In [2]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install opencv-python


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install scikit-image
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install pillow


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import numpy as np
from skimage.feature import canny
from skimage.measure import label, regionprops
from skimage.transform import hough_line

# histogram
def compute_histogram(image):
  hist = cv2.calcHist([image], [0], None, [256], [0, 256])
  return hist.flatten()

def edge_detection_canny(image):
    edges = canny(image)
    return edges.flatten()

# def image_smoothing(image):
#   smooth_image = cv2.GaussianBlur(image, (5,5), 0)
#   return smooth_image.flatten()

# def line_detection_hough(image):
#   edges = canny(image)
#   hspace, angles, dists = hough_line(edges)
#   hspace_flatten = hspace.flatten()
#   return hspace_flatten

# def connected_components(image):
#   labelled_image = label(image)
#   props = regionprops(labelled_image)
#   return [prop.area for prop in props]

def compute_sift(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors.flatten() if descriptors is not None else np.array([])

def pad_or_truncate(features, length):
    if len(features) >= length:
        return features[:length]
    else:
        return np.pad(features, (0, length - len(features)), 'constant')

# def compute_surf(image):
#     surf = cv2.SURF_create()
#     keypoints, descriptors = surf.detectAndCompute(image, None)
#     return descriptors.flatten() if descriptors is not None else np.array([])

In [21]:
import cv2

# Set a fixed length for the feature vectors
FIXED_FEATURE_LENGTH = 500
# function to extract features from all the images
def extract_features(images):
  features = []

  # loop through each image
  for image in images:
    # new_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    hist_features = compute_histogram(image)
    # hist_eq_features = histogram_equalization(image)
    # smooth_features = image_smoothing(image)
    # sobel_features = edge_detection_sobel(image)
    canny_features = edge_detection_canny(image)
    # hough_features = line_detection_hough(image)

    # connected_features = connected_components(image)
    sift_features = compute_sift(image)
    # surf_features = compute_surf(new_image)

    # combine the features in different set
    combined_features = np.concatenate([hist_features, sift_features, canny_features])
    # mid_level_features = np.concatenate([smooth_features, connected_features])

    # Pad or truncate to the fixed length
    combined_features = pad_or_truncate(combined_features, FIXED_FEATURE_LENGTH)

    # all_features = np.concatenate([combined_features, mid_level_features])
    features.append(combined_features)

  return np.array(features)


In [8]:
import os
print(os.getcwd())

C:\Users\Ritesh


In [9]:
from PIL import Image
import numpy as np
import os
import cv2

data_dir = r'C:\Users\Ritesh\Downloads\dataset_1\dataset_full'
categories = ['Building', 'Glacier', 'Sea', 'Mountains', 'Streets', 'Forest']
# Create the sharpening kernel 
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) 

def load_images(data_dir, categories):
    images = []
    labels = []
    for label, category in enumerate(categories):
        category_dir = os.path.join(data_dir, category)
        for files in os.listdir(category_dir):
            if files.endswith('.jpg'):
                file_path = os.path.join(category_dir, files)
                new_image = cv2.imread(file_path)
                sharpened_image = cv2.filter2D(new_image, -1, kernel) 
                if sharpened_image is not None:
                    images.append(sharpened_image)
                    labels.append(label)
                else:
                    print(f"Error reading image: {file_path}")
    return images, labels

# Load images and labels
images, labels = load_images(data_dir, categories)
print(len(images))

5245


In [10]:
from sklearn.model_selection import train_test_split

# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Extract features from training and testing images
X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)

X_train_features
X_test_features

array([[2.917e+03, 2.900e+01, 3.200e+01, ..., 3.800e+01, 1.700e+01,
        4.000e+00],
       [6.906e+03, 4.800e+01, 4.300e+01, ..., 8.800e+01, 2.800e+01,
        6.000e+00],
       [9.840e+02, 1.800e+01, 8.000e+00, ..., 3.000e+00, 0.000e+00,
        2.000e+00],
       ...,
       [8.645e+03, 6.000e+01, 7.400e+01, ..., 4.400e+01, 9.700e+01,
        1.400e+01],
       [9.044e+03, 5.700e+01, 5.600e+01, ..., 2.600e+01, 7.500e+01,
        1.110e+02],
       [3.410e+03, 5.100e+01, 5.200e+01, ..., 1.100e+01, 1.030e+02,
        4.900e+01]], dtype=float32)

In [11]:
from imblearn.over_sampling import SMOTE
# 
# Balance the training dataset using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_features, y_train)

X_train_resampled.shape
# y_train_resampled

(13224, 500)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train and Evaluate the k-NN Model
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, n_jobs=-1, verbose=2)
clf.fit(X_train_resampled, y_train_resampled)

y_pred = clf.predict(X_test_features)

print("Best parameters found: ", clf.best_params_)
print("Accuracy:", clf.best_score_)
print("Accuracy:", accuracy_score(y_test, y_pred)*100)
print(classification_report(y_test, y_pred, target_names=categories))
print(confusion_matrix(y_test, y_pred))

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Accuracy: 0.950317604355717
Accuracy: 53.86081982840801
              precision    recall  f1-score   support

    Building       0.23      0.29      0.25       112
     Glacier       0.35      0.38      0.37       105
         Sea       0.30      0.36      0.33       108
   Mountains       0.29      0.30      0.29        92
     Streets       0.21      0.34      0.26        91
      Forest       0.94      0.73      0.82       541

    accuracy                           0.54      1049
   macro avg       0.39      0.40      0.39      1049
weighted avg       0.62      0.54      0.57      1049

[[ 32  11  22  19  21   7]
 [ 12  40  27  13  12   1]
 [  8  20  39  25  14   2]
 [ 20  19  20  28   3   2]
 [ 28   9   4   7  31  12]
 [ 39  15  17   6  69 395]]


In [15]:
y_pred

for pred in y_pred:
    print(categories[pred])

Mountains
Forest
Sea
Sea
Glacier
Streets
Building
Glacier
Forest
Sea
Forest
Building
Building
Forest
Glacier
Building
Building
Streets
Forest
Forest
Streets
Forest
Building
Streets
Streets
Sea
Forest
Streets
Streets
Forest
Forest
Sea
Sea
Forest
Streets
Streets
Glacier
Sea
Streets
Sea
Building
Streets
Sea
Forest
Streets
Streets
Streets
Streets
Glacier
Forest
Forest
Building
Streets
Streets
Forest
Glacier
Sea
Building
Forest
Forest
Forest
Streets
Forest
Sea
Sea
Glacier
Building
Forest
Building
Streets
Sea
Forest
Sea
Glacier
Forest
Sea
Forest
Forest
Glacier
Building
Forest
Forest
Forest
Sea
Mountains
Mountains
Forest
Forest
Sea
Forest
Forest
Forest
Forest
Forest
Forest
Forest
Streets
Streets
Building
Glacier
Forest
Forest
Forest
Building
Forest
Forest
Forest
Streets
Forest
Sea
Forest
Mountains
Streets
Forest
Glacier
Building
Forest
Building
Sea
Sea
Sea
Forest
Forest
Glacier
Forest
Sea
Building
Sea
Streets
Mountains
Forest
Sea
Sea
Sea
Forest
Forest
Forest
Mountains
Building
Glacier
Forest


In [31]:
data_dir = r'C:\Users\Ritesh\Downloads\dataset_1\dataset_full\Building\1165.jpg'

cv_image = cv2.imread(data_dir)
# sharpened_image = cv2.filter2D(cv_image, -1, kernel) 

# cv2.imshow('Sharpened Image', sharpened_image)
# cv2.waitKey(0)  # Wait for any key press
# cv2.destroyAllWindows()  # Close all OpenCV windows
features = extract_features(cv_image)
prediction = clf.predict(features)
print(prediction)
print(len(prediction))


print(categories[prediction[0]])

TypeError: 'NoneType' object is not iterable

In [14]:
import joblib

# Save the best model to a file
joblib_file = "classification_model_1.pkl"  
joblib.dump(clf, joblib_file)


['classification_model_1.pkl']

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# training the random forest classifier
model = RandomForestClassifier(n_estimators=500, max_depth=30, random_state=42)
model.fit(X_train_features, y_train)

# make predictions on the test set
y_pred = model.predict(X_test_features)

In [67]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=categories))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.5319351763584366
              precision    recall  f1-score   support

    Building       0.00      0.00      0.00       112
     Glacier       0.00      0.00      0.00       105
         Sea       0.47      0.14      0.21       108
   Mountains       0.19      0.03      0.06        92
     Streets       1.00      0.01      0.02        91
      Forest       0.54      1.00      0.70       541

    accuracy                           0.53      1049
   macro avg       0.37      0.20      0.17      1049
weighted avg       0.43      0.53      0.39      1049

[[  0   0   2   2   0 108]
 [  1   0   4   4   0  96]
 [  0   0  15   6   0  87]
 [  0   0   6   3   0  83]
 [  0   0   4   0   1  86]
 [  0   0   1   1   0 539]]


C:\Users\Ritesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Ritesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Ritesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [68]:
import joblib

# Save the best model to a file
joblib_file = "best_classification_model.pkl"  
joblib.dump(model, joblib_file)

['best_classification_model.pkl']

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rfe = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=50)
X_train_reduced_rfe = rfe.fit_transform(X_train_resampled, y_train_resampled)
X_test_reduced_rfe = rfe.transform(X_test_features)

model_rfe = RandomForestClassifier(n_estimators=200, random_state=42)
model_rfe.fit(X_train_reduced_rfe, y_train_resampled)
y_pred_rfe = model_rfe.predict(X_test_reduced_rfe)
print("RFE Accuracy:", accuracy_score(y_test, y_pred_rfe))
print(classification_report(y_test, y_pred_rfe, target_names=categories))
print(confusion_matrix(y_test, y_pred_rfe))

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_scores = cross_val_score(RandomForestClassifier(n_estimators=500, random_state=42), X_train_reduced, y_train, cv=5)
print("Cross-validation scores:", cross_val_scores)
print("Mean cross-validation score:", cross_val_scores.mean())

In [ ]:
# model ensembling

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf1 = RandomForestClassifier(n_estimators=500, max_depth=30, random_state=42)
clf2 = SVC(probability=True)
clf3 = KNeighborsClassifier()
clf4 = GradientBoostingClassifier(random_state=42)

ensemble_model = VotingClassifier(estimators=[
    ('rf', clf1), 
    ('svc', clf2), 
    ('knn', clf3),
    ('gbc', clf4)
], voting='soft')

ensemble_model.fit(X_train_reduced, y_train)
y_pred_ensemble = ensemble_model.predict(X_test_reduced)


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# param_dist = {
#     'n_estimators': [100, 200, 300],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth': [None, 10, 20, 30],
#     # 'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42), param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
# random_search.fit(X_train_reduced, y_train)

# best_model = random_search.best_estimator_
# best_model

In [ ]:
# Predict on the test set
# y_pred = best_model.predict(X_test_reduced)
# y_pred

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate classification report
report = classification_report(y_test, y_pred, target_names=categories)
print("Classification Report:")
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate the model
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print(classification_report(y_test, y_pred_ensemble, target_names=categories))
print(confusion_matrix(y_test, y_pred_ensemble))

In [ ]:
import joblib

# Save the best model to a file
joblib_file = "best_model.pkl"  
joblib.dump(ensemble_model, joblib_file)
